# Training

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.pt')

train_results = model.train(
    data='data/dataset.yaml',
    epochs=40,
    device='cpu',  # Fix: Use CPU instead of MPS to avoid memory corruption
    workers=0,      # Fix: Single worker to avoid threading issues
    amp=False,      # Fix: Disable AMP for stability
    mosaic=0.0,     # daDisable data augmentation
    fliplr=0.0      # Disable horizontal flip
)

# Validation

In [ ]:
import numpy as np

from tqdm import tqdm

best = {"conf": None, "f1": -1, "metrics": None}
confs = np.linspace(0.005, 0.2, 40)
for conf in tqdm(confs):
    val = model.val(data="data/dataset.yaml", conf=float(conf), plots=False, verbose=False)
    f1 = val.box.f1.mean()
    if f1 > best["f1"]:
        best = {"conf": float(conf), "f1": float(f1)}

# Predict

In [ ]:
from ultralytics import YOLO
import cv2
from PIL import Image

model = YOLO(f'{train_results.save_dir}/weights/best.pt')

prediction_results = model.predict(
    # source='data/train/images/2dd1ddf2-IMG_1326.jpeg',
    source='data/test/images/03230a96-IMG_1332.jpeg',
    conf=best['conf'],
    project="test_overfitting",
    name="training_image_test"
)

result = prediction_results[0]

annot = result.plot(labels=False, conf=False)              # NumPy array with boxes drawn (BGR)

Image.fromarray(
    cv2.cvtColor(annot, cv2.COLOR_BGR2RGB)
)